In [1]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchain-core langchainhub
%pip install -U pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

index_name = 'tax-markdown-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")


llm = ChatOpenAI(model="gpt-4o")

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query = query, k=3)



prompt = ChatPromptTemplate.from_template("""
[Identity]
- 당신은 최고의 한국 세무 전문가입니다.
- [Context]를 참고하여 사용자의 질문에 답변하세요.

[Context]
{context}

[User Question]
{input}
""")

# LLM과 Prompt 준비
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# database는 PineconeVectorStore 같은 Retriever로 변환 가능해야 함
retriever = database.as_retriever(search_kwargs={"k": 4})

# Retrieval + Combine 연결
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# query -> 직장인 -> 거주자 chain 추가
dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""

  사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
  만약 사용자의 질문이 이미 명확하다면, 그대로 반환해주세요.
                                          
  [사전] : {dictionary}

  [사용자 질문] : {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

tax_chain = {"input" : dictionary_chain} | rag_chain

ai_response = tax_chain.invoke({"question": query})

In [ ]:
new_question = dictionary_chain.invoke({"question": query})

In [ ]:
new_question

In [ ]:
ai_response